In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/Tensor_decom_sample/data/'

In [ ]:
data=pd.read_csv(data_path + 'time_factor.csv')

In [ ]:
data.set_index(data.columns[len(data.columns)-1], inplace=True)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
weather_data = pd.read_csv(data_path + 'we_ev_el.csv')

In [ ]:
weather_data.shape

In [ ]:
weather_data.set_index('timestamp', inplace=True)
weather_data.head()

In [ ]:
preprocessed_data = pd.concat([data, weather_data.reindex(data.index)], axis=1)

In [ ]:
preprocessed_data

In [ ]:
import pandas as pd

In [ ]:
preprocessed_data.columns.tolist()

In [ ]:
data_ready = preprocessed_data.drop(columns=['precipitation','date','sunrise_datetime','sunset_datetime','event_start_date','event_end_date','event_start_date','event_location'])

In [ ]:
data_ready.rename(columns={"encoding": "daytime"}, inplace=True)

In [ ]:
data_ready

In [ ]:
# South East limits
se_limit_lat = 51.523811 # everthing lower than this value is south east
se_limit_lon = -0.101342 # everything greater than this value is south east

# East limits
e_limit_lat = 51.523811 # everything greater than this value is east
e_limit_lon = -0.022306 # everything greater than this is east

# South West limits
sw_limit_lat = 51.501631 # everthing lower than this value is south east
sw_limit_lon = -0.101342 # everything lower than this value is south east

# West limits
w_limit_lat = 51.501072  # everthing greater than this value is south east
w_limit_lon = -0.182406  # everthing lower than this value is south east

# Central limits
c_toplimit_lat = 51.501072 # everthing greater than this value is central
c_lowlimit_lat = 51.523811 # everything lower than this value is central

c_toplimit_lon = -0.101342 # everything lower than this value is central
c_lowlimit_lon = -0.182406 # everthing greater than this value is central

# North limits
n_toplimit_lon = -0.022306 # everything lower than this value is central
n_lowlimit_lon = -0.182406 # everthing greater than this value is central

n_lowlimit_lat = 51.523811 # everthing greater than this value is central

In [ ]:
loc = []

In [ ]:
for hour in data_ready.index:
    if data_ready['event_latitude'][hour] > c_toplimit_lat and data_ready['event_latitude'][hour]< c_lowlimit_lat and data_ready['event_longitude'][hour]< c_toplimit_lon and data_ready['event_longitude'][hour]>c_lowlimit_lon:
        loc.append('Central') # Central
    elif data_ready['event_longitude'][hour] < n_toplimit_lon and data_ready['event_longitude'][hour] > n_lowlimit_lon and data_ready['event_latitude'][hour] > n_lowlimit_lat:
        loc.append('North') # North
    elif data_ready['event_longitude'][hour] < w_limit_lon and data_ready['event_latitude'][hour] > w_limit_lat:
        loc.append('West') # West
    elif data_ready['event_longitude'][hour] < sw_limit_lon and data_ready['event_latitude'][hour] < sw_limit_lat:
        loc.append('South_West')  # South_West
    elif data_ready['event_longitude'][hour] > e_limit_lon and data_ready['event_latitude'][hour] > e_limit_lat:
        loc.append('East') # East
    elif data_ready['event_longitude'][hour] > se_limit_lon and data_ready['event_latitude'][hour] < se_limit_lat:
        loc.append('South_East') # South East
    else:
        loc.append('Other')

In [ ]:
data_ready['london_zone'] = loc

In [ ]:
data_ready

In [ ]:
data_ready.event_title.replace(np.nan,'No_Event', inplace=True)

In [ ]:
data_ready

In [ ]:
data_ready.drop(columns=['event_latitude','event_longitude'], inplace=True)

In [ ]:
data_ready

In [ ]:
data_ready[['snowfall']].boxplot()

In [ ]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse = False)

In [ ]:
# Fit encoder
ohe.fit(data_ready[['daytime','event_title','london_zone']])

In [ ]:
# Transform the current "Street" column
ohe.transform(data_ready[['daytime','event_title','london_zone']]).shape

In [ ]:
data_ready[ohe.get_feature_names_out()] = 0

In [ ]:
data_ready[ohe.get_feature_names_out()]=ohe.transform(data_ready[['daytime','event_title','london_zone']])

In [ ]:
data_ready

In [ ]:
data_preproc = data_ready.drop(columns=['daytime','event_title','london_zone','daytime_nighttime',data_ready.columns[0]])

In [ ]:
data_preproc

In [ ]:
data_preproc.info(verbose=True)

In [ ]:
data_preproc.columns[:149]

In [ ]:
import seaborn as sns

In [ ]:
#Partial correlation
part_data =  data_preproc.drop(columns=data_preproc.columns[:150])
part_data

In [ ]:
data_preproc['elisabeth_line'] = data_preproc['elisabeth_line']*1

In [ ]:
data_preproc

In [ ]:
# Heatmap of pairwise correlations
correlation_matrix = part_data.corr()
column_names = correlation_matrix.columns
sns.heatmap(correlation_matrix, xticklabels=column_names, yticklabels=column_names,cmap= "bwr");

In [ ]:
y_columns = data_preproc.columns[:150]
y_columns

In [ ]:
X_columns = data_preproc.columns[150:]
X_columns

In [ ]:
y = data_preproc[y_columns]

In [ ]:
y

In [ ]:
X = data_preproc[X_columns]
X

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
X_train.shape

In [ ]:
X_train =  X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test =  y_test.to_numpy()

In [ ]:
X_train.shape , y_train.shape

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
X_train.shape , X_test.shape

In [ ]:
# 0- Imports
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten

In [ ]:
# 1- RNN Architecture
model = Sequential()
model.add(layers.LSTM(units=500, activation='tanh',input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(layers.Dense(200))
model.add(layers.Dense(150, activation="linear"))

In [ ]:
# 2- Compilation
model.compile(loss='mae',
              optimizer=Adam(learning_rate=0.01),metrics=['accuracy']) # very high lr so we can converge with such a small dataset

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=10, restore_best_weights=True)

# 3- Fit
history = model.fit(X_train, y_train, epochs=50, verbose=1, validation_split=0.2,callbacks=[es])

In [ ]:
# 4- Predict
model.evaluate(X_test,y_test)

In [ ]:
from matplotlib import pyplot

In [ ]:
# plot history
pyplot.plot(history.history['loss'][1:], label='train')
pyplot.plot(history.history['val_loss'][1:], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# 4- Predict
y_hat = model.predict(X_test)

In [ ]:
y_hat.shape

In [ ]:
data_preproc